# MNIST Classification with Logistic Regression

In [0]:
import tensorflow as tf

In [0]:
#Reset tensorflow graph
tf.reset_default_graph()

###Step 1 : Collect Data
Use MNIST data provided with tensorflow library.

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Get Training and Test Data

In [0]:
trainX = mnist.train.images
trainY = mnist.train.labels

In [0]:
testX = mnist.test.images
testY = mnist.test.labels

How many Training and Test Examples?

In [0]:
print('Training input features: ', trainX.shape)
print('Training target data: ', trainY.shape)

In [0]:
trainY[0]

In [0]:
print('Test input features: ', testX.shape)
print('Test target data: ', testY.shape)

Lets define some parameters

In [0]:
#Directory to save logs and graph, change it as needed
logs_path='/tmp/mnist/lr2'

#Learning rate
learning_rate = 0.03

#Number of input features - MNIST has 784 features
n_features = trainX.shape[1]

#Number of possible output classess - 10 for MNIST
n_classes = trainY.shape[1]

#Model name for storage
model_name = 'mnist_lr.ckpt'

#How many times all the data will be shown to model
training_epochs = 100

# Build the Graph

Input placeholders

In [0]:
with tf.name_scope('input'):
    
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input") 
    
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

Define Weights and Bias

In [0]:
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([n_features, n_classes]))  #784,10
    b = tf.Variable(tf.zeros([n_classes])) #10

Prediction    

In [0]:
with tf.name_scope("Output"):
    y = tf.nn.softmax(tf.matmul(x,W) + b)

Loss

In [0]:
with tf.name_scope('Loss'):
    #Cross Entropy Loss
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

GradientDescent Optimizer

In [0]:
with tf.name_scope('train'):    
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        cross_entropy)    

Model Accuracy

In [0]:
with tf.name_scope('Accuracy'):
    
    #Get the number with highest probability
    prediction = tf.argmax(y,1,name="Predict")
    
    #Compare prediction with actual
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    
    #Calculate mean accuracy across all data examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
                              name="accuracy")

Loss and Accuracy Logging

In [0]:
# Training loss and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)

#Test loss and accuracy
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

# create log writer object
writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

# Execute the Graph

In [0]:
#Start Graph execution
with tf.Session() as sess:
    
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # perform training cycles
    for epoch in range(training_epochs):
        
        # perform the operations we defined earlier on batch
        _,acc,loss = sess.run([train_op, training_accuracy,training_loss],
                              feed_dict={x: trainX, y_: trainY})
        
        #log training accuracy and loss
        writer.add_summary(acc, epoch)
        writer.add_summary(loss, epoch) 
        
        #Test loss and accuracy
        #Please note we are giving test data for session run
        acc,loss = sess.run([test_accuracy,test_loss],
                                   feed_dict={x: testX, y_: testY})
        
        writer.add_summary(acc, epoch)
        writer.add_summary(loss, epoch)
        
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch)
            print ("Test Accuracy: ", accuracy.eval(feed_dict={x: testX, 
                                                               y_: testY}))               
    #Create a Saver to save the graph
    saver = tf.train.Saver()
    saver.save(sess, logs_path + '/' + model_name)